# Configuração inicial

### Instalação de bibliotecas

In [2]:
!pip install pyserial

You should consider upgrading via the 'C:\Users\Amanda\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


### Importação de bibliotecas

In [2]:
import serial
import serial.tools.list_ports
import time
import csv

### Definição de Constantes

Códigos entre o Arduino e o computador:

In [14]:
HANDSHAKE = 0
VOLTAGE_REQUEST = 1;
ON_REQUEST = 2;
STREAM = 3;

Constantes de configuração

In [44]:
N = 10 #Número de pontos coletados

### Funções

#### Handshake entre o computador e o Arduino

In [10]:
def handshake_arduino(arduino, sleep_time=1, print_handshake_message=False, handshake_code=0):
    """Make sure connection is established by sending and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout

#### Converter byte em número

To convert the byte string to numbers, we need to strip the carriage return and newline and decode the byte string into the Python strings we are used to using the decode() method. Finally, we can split the resulting string at the comma to get the voltage and time point for each.

Arduino values from analog inputs are 10 bit unsigned integers, with values ranging from 0 to 1023. So, when the voltage is written out, it will be an integer, which we need to convert to a voltage, knowing that the voltage on the Arduino Uno goes from zero to five volts. The time stamps are in milliseconds, transmitted as 32-bit unsigned integers. So, we should convert the voltage number we receive to Volts. The function below parses a byte string sent from Arduino.

#### Converter byte em número

In [21]:
def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    t, periodo, V = raw.rstrip().split(",")

    return int(t), int(periodo), int(V) * 5 / 1023

#### Solicita a leitura do Arduino

Solicita a leitura de um único ponto:

In [18]:
def request_single_voltage(arduino):
    """Ask Arduino for a single data point"""
    # Ask Arduino for data
    arduino.write(bytes([VOLTAGE_REQUEST]))

    # Read in the data
    raw = arduino.read_until()

    # Parse and return
    return parse_raw(raw)

# Identificação da porta serial

Lista de portas abertas

In [6]:
ports = serial.tools.list_ports.comports()

# Take a look
ports

Identificação do dispositivo em cada porta

In [7]:
[port.manufacturer for port in ports]

['Arduino LLC (www.arduino.cc)']

Descobre o string da porta associada

In [8]:
port = ports[0].device

OBSERVAÇÃO IMPORTANTE:

When opening a connection with Python, you cannot have the Serial Monitor nor Serial Plotter of the Arduino IDE open, since they will keep the port busy and Python cannot communicate with Arduino.

Ou seja: A IDE do arduino precisa estar fechada.

# Leitura da porta serial

Abrindo a porta serial e fazendo um handshake:

In [19]:
# Open port
arduino = serial.Serial(port, baudrate=115200, timeout=1)

# Call the handshake function
handshake_arduino(arduino, print_handshake_message=True, handshake_code=HANDSHAKE)

Handshake message: Handshake message received.



Teste de leitura individual:

In [32]:
request_single_voltage(arduino)

(879964, 68008, 1.4760508308895406)

Coleta e criação de uma lista de pontos:

In [46]:
data = []

for i in range (0,N):
        linha = request_single_voltage(arduino)
        data.append(linha)

Visualização do resultado:

In [47]:
print("time (ms)    Ts (ms)    voltage (V)")

for linha in data:
    print(linha)

time (ms)    Ts (ms)    voltage (V)
(1559758, 36967, 1.4711632453567938)
(1559766, 8, 1.466275659824047)
(1559770, 4, 1.466275659824047)
(1559774, 4, 1.466275659824047)
(1559778, 4, 1.466275659824047)
(1559782, 4, 1.4613880742913001)
(1559786, 4, 1.4613880742913001)
(1559790, 4, 1.4613880742913001)
(1559794, 4, 1.4613880742913001)
(1559798, 4, 1.4613880742913001)


In [52]:
print(data)

[(1559758, 36967, 1.4711632453567938), (1559766, 8, 1.466275659824047), (1559770, 4, 1.466275659824047), (1559774, 4, 1.466275659824047), (1559778, 4, 1.466275659824047), (1559782, 4, 1.4613880742913001), (1559786, 4, 1.4613880742913001), (1559790, 4, 1.4613880742913001), (1559794, 4, 1.4613880742913001), (1559798, 4, 1.4613880742913001)]


Salvando em um arquivo CSV:

In [55]:
# opening the csv file in 'a+' mode
file = open('ColetaUSB.csv', 'a+', newline ='')

fields = ["time (ms)","Ts (ms)","voltage (V)"]

# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerow(fields)
    write.writerows(data)

Fechando a porta serial:

In [56]:
arduino.close()

# Referências

Leitura da porta serial:
https://colab.research.google.com/github/justinbois/bootcamp/blob/gh-pages/2021/lessons/l40_serial.ipynb#scrollTo=dFvTZJJYkHXE

Arquivo CSV:
https://www.geeksforgeeks.org/writing-data-from-a-python-list-to-csv-row-wise/?ref=lbp